In [1]:
import base64
import os 
import requests
import datetime 
from urllib.parse import urlencode 
import pprint

In [2]:
client_id = "b1f767d933f14dacab7f4062891921bb"
client_secret = "9207250566db44009edf27ca30c1caf3"


In [18]:
class SpotifyAPI(object):
    access_token = None;
    access_token_expires = datetime.datetime.now()
    access_token_did_expires = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self,client_id,client_secret):
        super().__init__()
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Returns A base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set the client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
        
    def get_token_headers(self):   
        client_creds_b64 = self.get_client_credentials()
        return {
        'Authorization':f"Basic {client_creds_b64}"
        }
    
    
    def get_token_data(self):
        return {
            'grant_type':"client_credentials"
        }
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_header = self.get_token_headers()
        r = requests.post(token_url,data = token_data,headers = token_header)
        print(r.json())
        if r.status_code not in range(200,299):
            raise Exception("Could not authenticate client.")
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds = expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expires = expires <  now
        return True
    
    def get_access_token(self):
        token= self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires <now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization":f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self,lookup_id,resource_type='albums',version="v1"):
#         example :-   GET https://api.spotify.com/v1/albums/{id}
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers = headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def get_album(self,_id):
        return self.get_resource(_id,resource_type='albums')
    
    def get_artist(self,_id):
        return self.get_resource(_id,resource_type="artists")
    
    def search(self,query,search_type='track'):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({'q':f'{query}','type':f'{search_type.lower()}'})

        final_endpoint = f"{endpoint}?{data}"

        r = requests.get(final_endpoint,headers =headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    


In [19]:
spotify = SpotifyAPI(client_id,client_secret)
spotify.search("Tu aake dekhle","Track")

# print(spotify.access_token_expires)

{'access_token': 'BQD9o_L8mHSrV6p5bBuIBrw_WBz6r9LhDPlXVVIBh1iCkr7vOiQxcaQXxnPNLF36XJ0oC_lpzQja9XJ1DUo', 'token_type': 'Bearer', 'expires_in': 3600}


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Tu+aake+dekhle&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5NHm4TU5Twz7owibYxJfFU'},
       'href': 'https://api.spotify.com/v1/artists/5NHm4TU5Twz7owibYxJfFU',
       'id': '5NHm4TU5Twz7owibYxJfFU',
       'name': 'King',
       'type': 'artist',
       'uri': 'spotify:artist:5NHm4TU5Twz7owibYxJfFU'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
      'DK',
      'D

In [12]:
access_token = spotify.access_token
print(access_token)

BQA2NUtopH7oQBTpjFYkIpgCXqWdnTClr5Yg3gsZuaMzG1lgaEGqPK7hyQv74Dq83iROhUgcJBPq1GmG2kQ


In [20]:
album_id = "7uftfPn8f7lwtRLUrEVRYM"
res = spotify.get_album("7uftfPn8f7lwtRLUrEVRYM")
res

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5NHm4TU5Twz7owibYxJfFU'},
   'href': 'https://api.spotify.com/v1/artists/5NHm4TU5Twz7owibYxJfFU',
   'id': '5NHm4TU5Twz7owibYxJfFU',
   'name': 'King',
   'type': 'artist',
   'uri': 'spotify:artist:5NHm4TU5Twz7owibYxJfFU'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AO',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BY',
  'BZ',
  'CA',
  'CH',
  'CI',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DJ',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JM',
  'JO',
  'JP',
  'KE',
  'KG',
  'KH',
  'KI

In [21]:
artist_id = "5NHm4TU5Twz7owibYxJfFU"
resp = spotify.get_artist(artist_id)
resp

{'external_urls': {'spotify': 'https://open.spotify.com/artist/5NHm4TU5Twz7owibYxJfFU'},
 'followers': {'href': None, 'total': 182131},
 'genres': ['desi hip hop', 'desi pop', 'hindi hip hop'],
 'href': 'https://api.spotify.com/v1/artists/5NHm4TU5Twz7owibYxJfFU',
 'id': '5NHm4TU5Twz7owibYxJfFU',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5ebbe8967150d8f0a942dcf02af',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174be8967150d8f0a942dcf02af',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178be8967150d8f0a942dcf02af',
   'width': 160}],
 'name': 'King',
 'popularity': 63,
 'type': 'artist',
 'uri': 'spotify:artist:5NHm4TU5Twz7owibYxJfFU'}